In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Numeric, DateTime, ForeignKey
from sqlalchemy.exc import ProgrammingError
import configparser
import secrets

In [2]:
#pegando os arquivos csv e criando DF;

clientes = pd.read_csv("olist_customers_dataset.csv") #dimensao feita;
itens_pedidos = pd.read_csv("olist_order_items_dataset.csv") 
pagamentos = pd.read_csv("olist_order_payments_dataset.csv") #dimensao feita;
pedidos = pd.read_csv("olist_orders_dataset.csv") #dimensao feita;
produtos_vendidos = pd.read_csv("olist_products_dataset.csv") #dimensao feita;
avaliacoes = pd.read_csv("olist_order_reviews_dataset.csv") 

### Configurando as DIM

#### CONFIGURANDO A DIM_PRODUTOS // LEMBRETE* LEVAR A PRODUTOS_NULL PARA O BD

In [4]:
#dimensao produtos
dim_products = produtos_vendidos
#removendo valores duplicados;
dim_products = dim_products.drop_duplicates(subset=['product_id'])
#ARMAZENANDO OS VALORES NULOS DA TABELA "dim_produtos" EM OUTRO DF;
products_null = produtos_vendidos[produtos_vendidos.isnull().any(axis=1)]
#removendo colunas que não entrarão na minha dimensao para análise;
dim_products = dim_products.drop(columns=['product_name_lenght','product_description_lenght','product_photos_qty','product_weight_g','product_length_cm','product_height_cm','product_width_cm'])
#removendo valores nulos;
dim_products = dim_products.dropna()

print(len(dim_products))
print(len(products_null))

32341
611


#### CONFIGURANDO A DIM_PEDIDOS // LEMBRETE* LEVAR A PEDIDOS_NULL PARA O BD

In [4]:
#dimensao pedidos
dim_orders = pedidos
# removendo os valores duplicados
dim_orders = dim_orders.drop_duplicates(subset=['order_id'])
#ARMAZENANDO OS VALORES NULOS DA TABELA "dim_pedidos" EM OUTRO DF;
orders_null = pedidos[pedidos.isnull().any(axis=1)]
#removendo colunas que não entraram na minha dimensão para análise;
dim_orders = dim_orders.drop(columns=['order_status','order_id','order_estimated_delivery_date','order_delivered_customer_date','order_delivered_carrier_date','order_approved_at'])
#removendo os valores nulos;
dim_orders = dim_orders.dropna()

print(len(dim_orders))
print(len(orders_null))

99441
2980


#### CONFIGURANDO A DIM_CLIENTES CONFERIR CUSTOMER_ID

In [5]:
#dimensao clientes;
dim_customers = clientes
#removendo valores nulos;
dim_customers = dim_customers.dropna()
#removendo valores duplicados;
dim_customers = dim_customers.drop_duplicates(subset=["customer_unique_id"])
#removendo a coluna customer_id  pois não irei usar nessa dimensão;
dim_customers = dim_customers.drop(columns=['customer_id'])

print(len(dim_customers))

96096


#### CONFIGURANDO A DIM_PAGAMENTOS  // LEMBRETE* LEVAR A  PAGAMENTOS_NULL PARA O BD

In [4]:
#dimensão pagamentos; copy() resolve problema com atribuição simples... o DataFrame pagamentos permanece inalterado após modificações em dim_pagamentos...
dim_payments = pagamentos.copy()
#removendo dados duplicados
dim_payments = dim_payments.dropna()
#CRIANDO UM NOVO ID para referenciar na fato e não ter problema com o ID duplicado na fato;
dim_payments['payments_id'] = dim_payments['payments_id'] = range(1, len(dim_payments) + 1)
# Somando o valor final de pagamento e agrupando pelo tipo de pagamento e armazenando em outra coluna;
dim_payments['value_type_payment'] = dim_payments.groupby(['order_id','payment_type'])['payment_value'].transform('sum')
# Somando a quantidade de vezes que o pagamento foi feito de acordo com o tipo de pagamento e armazenando em outra coluna;
dim_payments['payment_sequential_per_type'] = dim_payments.groupby(['order_id','payment_type'])['payment_sequential'].transform('count')
#excluindo tabela de payment_value e payment_sequential pois não será mais usada devido a coluna value_type_payment e payment_installments pois estará na fato;
dim_payments = dim_payments.drop(columns=['payment_sequential','payment_value','payment_installments'])
#CRIANDO UM ID UNICO PARA PODER REFERENCIAR NA MINHA TABELA FATO; USANDO A BIBLIOTECA SECRETS dim_payments['payments_id'] = [secrets.token_hex(16) for _ in range(len(dim_payments))]
#dim_payments['payments_id'].apply(lambda x: x[:32]) FAZENDO UMA FUNÇÃO LAMBDA PARA QUE A CHAVE TENHA UM TOTAL DE 32DIGITOS, PARA SEGUIR O PADRÃO DOS ID's;
#excluindo as informações duplicadas; 
dim_payments = dim_payments.drop_duplicates(subset=['order_id','payment_type'])

#conferindo resultado;
display(dim_payments[dim_payments['order_id'] == 'ccf804e764ed5650cd8759557269dc13'])
display(dim_payments[dim_payments['order_id'] == '8272b63d03f5f79c56e9e4120aec44ef'])

,order_id,payment_type,payments_id,value_type_payment,payment_sequential_per_type
4346,ccf804e764ed5650cd8759557269dc13,voucher,4347,60.48,25
14362,ccf804e764ed5650cd8759557269dc13,credit_card,14363,2.20,1


,order_id,payment_type,payments_id,value_type_payment,payment_sequential_per_type
71010,8272b63d03f5f79c56e9e4120aec44ef,credit_card,71011,196.11,1


In [3]:
display(itens_pedidos)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


### Configurando a Fato

In [3]:
#Iniciando a tabela FATO com a junção das tabelas pedidos e pagamentos;
fato_sales = pd.merge(pedidos,pagamentos, on='order_id',how='left')
#Excluindo colunas que não estará na minha análise; 
fato_sales = fato_sales.drop(columns=['order_status','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date'])
#criando uma coluna temporaria para agrupar o valor do pedido pelo tipo de pagamento;
fato_sales['temp_type_payment'] = fato_sales.groupby(['order_id','payment_type'])['payment_value'].transform('sum')

fato_sales['paid_value'] = fato_sales.groupby(['order_id'])['payment_value'].transform('sum') 

#fato_sales['price_total'] = fato_sales.groupby(['order_id'])['value_type_payment'].transform('sum')
fato_sales = fato_sales.drop(columns=['payment_sequential','payment_type','payment_value','temp_type_payment'])

fato_sales = fato_sales.drop_duplicates()

In [4]:
# Unindo a fato com a tabela de itens_pedidos;
fato_sales = pd.merge(fato_sales,itens_pedidos, on='order_id',how='left')

# Excluindo colunas que não estará na minha análise;
fato_sales = fato_sales.drop(columns=['seller_id','shipping_limit_date','order_item_id'])

# Criando uma coluna com o valor final do produto por pedido;
fato_sales['price_total'] = fato_sales.groupby(['order_id','product_id'])['price'].transform('sum')

# Criando uma coluna com o valor final do frete por pedido;
fato_sales['freight_total'] = fato_sales.groupby(['order_id','product_id'])['freight_value'].transform('sum')

# Contando a quantidade produtos por pedido e armazenando esse valor em outra coluna; decisão minha para evitar a quantidade de linhas que teria com a coluna "order_item_id"  que é a sequencia que um produto se repete no pedido, agrupei todos os produtos e deixei a quantidade final;
fato_sales['itens_per_order'] = fato_sales.groupby(['order_id','product_id'])['product_id'].transform('count')

# excluindo as linhas duplicadas;
fato_sales = fato_sales.drop_duplicates(subset=['product_id','order_id'])

In [5]:
#unido a tabela de avaliacoes à fato;
fato_sales = pd.merge(fato_sales,avaliacoes, on='order_id',how='left')

#removendo colunas que não usarei na análise;
fato_sales = fato_sales.drop(columns=['review_answer_timestamp','review_creation_date','review_comment_message','review_comment_title'])

In [6]:
# Unindo a tabela de produtos à fato;
fato_sales = pd.merge(fato_sales,produtos_vendidos, on='product_id',how='left')

#removendo colunas que não usarei na análise;
fato_sales = fato_sales.drop(columns=['product_category_name','product_name_lenght','product_description_lenght','product_photos_qty','product_weight_g','product_length_cm','product_height_cm','product_width_cm'])

In [7]:
#unido a tabela de clientes à fato;
fato_sales = pd.merge(fato_sales,clientes, on='customer_id',how='left')

#removendo as colunas de acordo com a modelagem;
fato_sales = fato_sales.drop(columns=['customer_zip_code_prefix','customer_city','customer_state'])

In [8]:
'''#pegando as linhas nulas na coluna de categoria de produto e armazenando em outro DF;
fato_product_category_null = fato[fato['product_category_name'].isna()]
#removendo da tabela fato as linhas nulas da coluna "product_category_name";
fato = fato[~fato['product_category_name'].isna()]'''

'#pegando as linhas nulas na coluna de categoria de produto e armazenando em outro DF;\nfato_product_category_null = fato[fato[\'product_category_name\'].isna()]\n#removendo da tabela fato as linhas nulas da coluna "product_category_name";\nfato = fato[~fato[\'product_category_name\'].isna()]'

In [9]:
#pegando as linhas nulas na parte do review e armazenando em outro DF;
fato_review_null = fato_sales[fato_sales['review_id'].isna()]
#removendo da tabela fato as linhas nulas da coluna review_id
fato_sales = fato_sales[~fato_sales['review_id'].isna()]

In [10]:
installments_null = fato_sales[fato_sales.isnull().any(axis=1)]
fato_sales = fato_sales[~fato_sales['payment_installments'].isna()]

In [20]:
#display(fato_sales[fato_sales['order_id'] == 'ccf804e764ed5650cd8759557269dc13'])
display(fato_sales[fato_sales['order_id'] == 'df56136b8031ecd28e200bb18e6ddb2e'])
display(fato_sales[fato_sales['order_id'] == 'c88b1d1b157a9999ce368f218a407141'])

,order_id,customer_id,payment_installments,paid_value,product_id,price,freight_value,price_total,freight_total,itens_per_order,review_id,review_score,customer_unique_id
34575,df56136b8031ecd28e200bb18e6ddb2e,b7770073b02ed1d626a027ce86a4ff82,1.0,204.72,f28cbd414cd06f1ae84065c8dd8be834,23.90,2.53,23.90,2.53,1.0,c444278834184f72b1484dfe47de7f97,5.0,2e43e031f10de28e557c35ef668f9396
34576,df56136b8031ecd28e200bb18e6ddb2e,b7770073b02ed1d626a027ce86a4ff82,1.0,204.72,f28cbd414cd06f1ae84065c8dd8be834,23.90,2.53,23.90,2.53,1.0,72a1098d5b410ae50fbc0509d26daeb9,5.0,2e43e031f10de28e557c35ef668f9396
34577,df56136b8031ecd28e200bb18e6ddb2e,b7770073b02ed1d626a027ce86a4ff82,1.0,204.72,f28cbd414cd06f1ae84065c8dd8be834,23.90,2.53,23.90,2.53,1.0,44f3e54834d23c5570c1d010824d4d59,5.0,2e43e031f10de28e557c35ef668f9396
34578,df56136b8031ecd28e200bb18e6ddb2e,b7770073b02ed1d626a027ce86a4ff82,1.0,204.72,50aa8f292a9510d5542f2a078903a2a7,63.92,15.82,63.92,15.82,1.0,c444278834184f72b1484dfe47de7f97,5.0,2e43e031f10de28e557c35ef668f9396
34579,df56136b8031ecd28e200bb18e6ddb2e,b7770073b02ed1d626a027ce86a4ff82,1.0,204.72,50aa8f292a9510d5542f2a078903a2a7,63.92,15.82,63.92,15.82,1.0,72a1098d5b410ae50fbc0509d26daeb9,5.0,2e43e031f10de28e557c35ef668f9396
34580,df56136b8031ecd28e200bb18e6ddb2e,b7770073b02ed1d626a027ce86a4ff82,1.0,204.72,50aa8f292a9510d5542f2a078903a2a7,63.92,15.82,63.92,15.82,1.0,44f3e54834d23c5570c1d010824d4d59,5.0,2e43e031f10de28e557c35ef668f9396
34581,df56136b8031ecd28e200bb18e6ddb2e,b7770073b02ed1d626a027ce86a4ff82,1.0,204.72,3025303c80b01a0926fda2841cc31b4a,66.90,31.65,66.90,31.65,1.0,c444278834184f72b1484dfe47de7f97,5.0,2e43e031f10de28e557c35ef668f9396
34582,df56136b8031ecd28e200bb18e6ddb2e,b7770073b02ed1d626a027ce86a4ff82,1.0,204.72,3025303c80b01a0926fda2841cc31b4a,66.90,31.65,66.90,31.65,1.0,72a1098d5b410ae50fbc0509d26daeb9,5.0,2e43e031f10de28e557c35ef668f9396
34583,df56136b8031ecd28e200bb18e6ddb2e,b7770073b02ed1d626a027ce86a4ff82,1.0,204.72,3025303c80b01a0926fda2841cc31b4a,66.90,31.65,66.90,31.65,1.0,44f3e54834d23c5570c1d010824d4d59,5.0,2e43e031f10de28e557c35ef668f9396


,order_id,customer_id,payment_installments,paid_value,product_id,price,freight_value,price_total,freight_total,itens_per_order,review_id,review_score,customer_unique_id
38767,c88b1d1b157a9999ce368f218a407141,ae0fb7b01c548c4fd30f19f55453ec4a,4.0,42.77,b1acb7e8152c90c9619897753a75c973,34.99,7.78,34.99,7.78,1.0,ffb8cff872a625632ac983eb1f88843c,3.0,b64ebaf3d11b7209fe566364cc359a51
38768,c88b1d1b157a9999ce368f218a407141,ae0fb7b01c548c4fd30f19f55453ec4a,4.0,42.77,b1acb7e8152c90c9619897753a75c973,34.99,7.78,34.99,7.78,1.0,202b5f44d09cd3cfc0d6bd12f01b044c,5.0,b64ebaf3d11b7209fe566364cc359a51
38769,c88b1d1b157a9999ce368f218a407141,ae0fb7b01c548c4fd30f19f55453ec4a,4.0,42.77,b1acb7e8152c90c9619897753a75c973,34.99,7.78,34.99,7.78,1.0,fb96ea2ef8cce1c888f4d45c8e22b793,5.0,b64ebaf3d11b7209fe566364cc359a51


#### CRIANDO O BANCO DE DADOS

In [15]:
#função para ler o arquivo database.ini onde armazena as credenciais;
def read_db_config(filename='database.ini', section='postgresql'):
    parser = configparser.ConfigParser()
    parser.read(filename)

    db_conf = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db_conf[param[0]] = param[1]
    return db_conf

db_conf = read_db_config()

#URL de conexão ao postgres usando o create_engine e usando as credenciais no database.ini (adiantando as melhores praticas de conexão)
engine_url = f"postgresql://{db_conf['user']}:{db_conf['password']}@{db_conf['host']}:{db_conf['port']}/postgres"
engine = create_engine(engine_url)

In [16]:
# instanciando e nomeando o BD que vou usar;
db_name = 'desafio_sqlalchemy'

In [17]:
# CONEXÃO E CRIAÇÃO DO NOVO BANCO DE DADOS;
# a utilização do With permite que a conexão seja fechada automaticamente; então não preciso usar o conn.close() dessa forma;
with engine.connect() as conn:
    conn = conn.execution_options(isolation_level="AUTOCOMMIT")
    try:
        conn.execute(f"CREATE DATABASE {db_name}")
        print(f"Banco de dados '{db_name}' criado com sucesso!")
    except ProgrammingError as e:
        print(f"Erro ao criar o banco de dados: {e}")
    #finally:   #DEIXANDO ESSE PARTE DO CÓDIGO PARA LEMBRAR DESSA OPÇÃO;
        #if conn is not None:
            #print(f"Conexão com o banco de dados '{db_name}' fechada!")
            #conn.close()

Banco de dados 'desafio_sqlalchemy' criado com sucesso!


#### CRIANDO AS TABELAS NO BANCO QUE FOI FEITO NO CODIGO ACIMA

In [18]:
#criando uma nova url de conexão agora para acessar o novo banco de dados;
new_engine_url = f"postgresql://{db_conf['user']}:{db_conf['password']}@{db_conf['host']}:{db_conf['port']}/{db_name}"
new_engine = create_engine(new_engine_url)

In [19]:
# Criar tabelas no novo banco de dados
metadata = MetaData()

# Definição das tabelas
tabela_clientes = Table('dim_customers', metadata,
                       Column('customer_unique_id', String(33), primary_key=True),
                       Column('customer_zip_code_prefix', Integer),
                       Column('customer_city', String(100)),
                       Column('customer_state', String(2))
                       )

tabela_pagamentos = Table('dim_payments', metadata,
                       Column('order_id', String(33), primary_key=True),
                       Column('payment_type', String(25), primary_key=True),
                       Column('value_type_payment', Numeric(10,2)),
                       Column('payment_sequential_per_type', Integer)
                       )

tabela_pedidos = Table('dim_orders', metadata,
                       Column('customer_id', String(33), primary_key=True),
                       Column('customer_purchase_timestamp', DateTime)
                       )

tabela_products = Table('dim_products', metadata,
                       Column('product_id', String(33), primary_key=True),
                       Column('product_category_name', String(50))
                       )
tabela_fato = Table('fato_sales', metadata,
                       Column('order_id', String(33), ForeignKey('dim_payments.order_id'), primary_key=True),
                       Column('customer_id', String(33), ForeignKey('dim_orders.customer_id'), primary_key=True),
                       Column('review_id', String(33), primary_key=True),
                       Column('customer_unique_id', String(33), ForeignKey('dim_customers.customer_unique_id'), primary_key=True),
                       Column('product_id', String(33), ForeignKey('dim_products.product_id'), primary_key=True),
                       Column('price', Numeric(10,2)),
                       Column('freight_value', Numeric(10,2)),
                       Column('itens_per_order', Integer),
                       Column('price_total', Numeric(10,2)),
                       Column('freight_total', Numeric(10,2)),
                       Column('paid_value', Numeric(10,2)),
                       Column('payment_installments', String(2)),
                       Column('score_review', String(1))                                        
                       )


In [20]:
# Conectar ao novo banco de dados, criar as tabelas e alimentar com os dataframes;
with new_engine.connect() as conn:
    try:
        metadata.create_all(new_engine)
        print("Tabelas criadas com sucesso!")
        dim_customers.to_sql('dim_customers', con=new_engine, if_exists='append', index=False)
        dim_payments.to_sql('dim_payments', con=new_engine, if_exists='append', index=False)
        dim_orders.to_sql('dim_orders', con=new_engine, if_exists='append', index=False)
        dim_products.to_sql('dim_products', con=new_engine, if_exists='append', index=False)
        fato_sales.to_sql('fato_sales', con=new_engine, if_exists='append', index=False)
        #LEMBRETE FALTA CRIAR AS TABELAS COM OS DADOS NULOS
        print("Dados inseridos com sucesso!")
    except ProgrammingError as err:
        print(f"Erro ao inserir dados na tabela{tabela_clientes}: {err}")

Erro ao inserir dados na tabeladim_customers: (psycopg2.errors.InvalidForeignKey) there is no unique constraint matching given keys for referenced table "dim_payments"

[SQL: 
CREATE TABLE fato_sales (
	order_id VARCHAR(33) NOT NULL, 
	customer_id VARCHAR(33) NOT NULL, 
	review_id VARCHAR(33) NOT NULL, 
	customer_unique_id VARCHAR(33) NOT NULL, 
	product_id VARCHAR(33) NOT NULL, 
	price NUMERIC(10, 2), 
	freight_value NUMERIC(10, 2), 
	itens_per_order INTEGER, 
	price_total NUMERIC(10, 2), 
	freight_total NUMERIC(10, 2), 
	paid_value NUMERIC(10, 2), 
	payment_installments VARCHAR(2), 
	score_review VARCHAR(1), 
	PRIMARY KEY (order_id, customer_id, review_id, customer_unique_id, product_id), 
	FOREIGN KEY(order_id) REFERENCES dim_payments (order_id), 
	FOREIGN KEY(customer_id) REFERENCES dim_orders (customer_id), 
	FOREIGN KEY(customer_unique_id) REFERENCES dim_customers (customer_unique_id), 
	FOREIGN KEY(product_id) REFERENCES dim_products (product_id)
)

]
(Background on this error a

In [ ]:
engine.dispose()
new_engine.dispose()

In [ ]:
display(fato_sales[fato_sales['order_id'] == 'df56136b8031ecd28e200bb18e6ddb2e'])

In [20]:
fato_sales = fato_sales.drop(columns=['review_id','review_score'])
fato_sales = fato_sales.drop_duplicates()

In [25]:
display(fato_sales[fato_sales['order_id'] == '566f53fc7d36fa366f7e221468121877'])

,order_id,customer_id,payment_installments,paid_value,product_id,price,freight_value,price_total,freight_total,itens_per_order,customer_unique_id
44024,566f53fc7d36fa366f7e221468121877,83b52613d1bcd8c6a66b41526405bad6,1.0,85.6,02a99f266fc960423c1186ecbd627804,69.9,15.7,69.9,15.7,1.0,2e43e031f10de28e557c35ef668f9396


#####  TALVEZ SEJA INTERESSANTE

In [ ]:
#armazenando os pedidos de acordo com o seu status;
#NÃO PRECISO LEVAR ISSO;
'''# Armazenando os pedidos cancelados, enviados, indisponíveis e faturados;
order_canceled = fato[fato['order_status'] == 'canceled']
order_shipped = fato[fato['order_status'] == 'shipped']
order_unavailable = fato[fato['order_status'] == 'unavailable']
order_invoiced = fato[fato['order_status'] == 'invoiced']

# Removendo todas as linhas onde os pedidos estão cancelados, enviados, indisponíveis e faturados;
fato = fato[~fato['order_status'].isin(['canceled', 'shipped', 'unavailable', 'invoiced'])]
#Excluindo colunas que não estará na minha análise; 
fato = fato.drop(columns=['order_status'])'''

In [16]:
'''# QUANDO QUISER SABER ONDE OS IDS SE REPETEM
# Passo 1: Identificar os review_id duplicados
#duplicated_order_ids = avaliacoes[avaliacoes.duplicated('review_id', keep=False)]['review_id']
# Passo 2: Filtrar as linhas que possuem esses order_id duplicados
#duplicated_rows = avaliacoes[avaliacoes['review_id'].isin(duplicated_order_ids)]'''

In [17]:
'''#### CONFIGURANDO A DIM_ITENS_POR_PRODUTOS /// ESTARÁ NA FATO!!!!
#dim_itens_por_pedidos = itens_pedidos

#### contando a quantidade produtos por pedidos e armazenando esse valor em outra coluna 
#dim_itens_por_pedidos['itens_per_order'] = dim_itens_por_pedidos.groupby(['order_id','product_id'])['product_id'].transform('count')

##### excluindo as linhas iguais "drop_duplicates" ; # NA FATO, DROP DESSA FORMA
#dim_itens_por_pedidos = dim_itens_por_pedidos.drop_duplicates(subset=['product_id', 'order_id'])

##### removendo a coluna ordem_item_id, pois depois de ter feito a coluna com a quantidade dos produtos em cada pedido ela não é necessária e também removendo as colunas que estarão na FATO;
#dim_itens_por_pedidos = dim_itens_por_pedidos.drop(columns=['order_item_id','price','freight_value'])

#####depois de tirar a dúvida, decidir se essas colunas também serão removidas;
#dim_itens_por_pedidos = dim_itens_por_pedidos.drop(columns=['seller_id','shipping_limit_date'])

#####imprimindo para visualizar;
#display(dim_itens_por_pedidos[dim_itens_por_pedidos['order_id'] == '8272b63d03f5f79c56e9e4120aec44ef'])'''

In [18]:
'''# criação que deve ser feita na tabela fato
### AQUI ESTOU FAZENDO OS TESTES PELA TABELA "DIM";

#####somando os valores dos produtos por pedido e o valor total do frete por item em cada pedido, agrupando pelo id do pedido e armazenando o resultado em duas novas colunas respectivamente;
#teste = itens_pedidos.groupby('order_id')[['price','freight_value']].sum().reset_index()

#####renomeando as colunas price e freight_value para price_total e freight_total pois agora são esses valores.
#teste.rename(columns={'price': 'price_total','freight_value':'freight_total'}, inplace=True)

#####criando a coluna paid_value, que é a soma do preço do total dos itens + o preço total do frete.
#teste['paid_value'] = teste['price_total'] + teste['freight_total']

#####testando para conferir o resultado; do codigo acima
#display(teste[teste['order_id'] == '8272b63d03f5f79c56e9e4120aec44ef'])'''

In [ ]:
'''#configuração de acesso ao postgres e alimentação das tabelas;

import psycopg2
from sqlalchemy import create_engine
import configparser

#função para ler o arquivo database.ini onde armazena as credenciais;
def read_db_config(filename='database.ini', section='postgresql'):
    parser = configparser.ConfigParser()
    parser.read(filename)

    db_conf = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db_conf[param[0]] = param[1]
    return db_conf

db_conf = read_db_config()

connection_string = f"postgresql+psycopg2://{db_conf['user']}:{db_conf['password']}@{db_conf['host']}:{db_conf['port']}/{db_conf['database']}"

engine = create_engine(connection_string)

#alimentando as tabelas existentes no banco de dados através dos dataframes;
table_name = "dim_avaliacoes"
dim_avaliacoes.to_sql(table_name, engine, if_exists='append', index=False)

table_name = "dim_clientes"
dim_clientes.to_sql(table_name, engine,if_exists='append', index=False)

table_name = "dim_pagamentos"
dim_pagamentos.to_sql(table_name, engine,if_exists='append', index=False)

table_name = "dim_pedidos"
dim_pedidos.to_sql(table_name, engine,if_exists='append', index=False)

table_name = "dim_produtos"
dim_produtos.to_sql(table_name, engine,if_exists='append', index=False)

engine.dispose()'''